In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Load data using pandas
bitcoin_url = 'https://raw.githubusercontent.com/RDeconomist/observatory/main/Bitcoin%20Price.csv'
bitcoin_data = pd.read_csv(bitcoin_url)
date = bitcoin_data['Date']
price = bitcoin_data['Closing Price (USD)'].astype(float)
print(f'Date range: {date.iloc[0]} to {date.iloc[-1]}')
print(f'Min 1BTC=${??}; Max 1BTC=${??}')
print(f'Min price occurred on {??}. Max price occurred on {??}')

plt.plot(price)
plt.ylabel('Bitcoin Price [$]')
plt.xticks(range(0, len(date), 300), date.iloc[::300], rotation=30)
plt.show()

In [ ]:
class TimeSeriesDataset(keras.utils.Sequence):
    def __init__(self, time_series, sequence_length):
        self.time_series = time_series
        self.sequence_length = sequence_length
        self._index = 0
    def __len__(self):
        return len(self.time_series) - self.sequence_length


    def __getitem__(self, idx):
        sequence = self.time_series[idx : idx + self.sequence_length]
        return sequence

    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)


print('=== Dataset ===')
dataset = TimeSeriesDataset(??)
t_idx = ??
print(f"Prices starting on {date[t_idx]}:\n ", dataset[t_idx])

## Linear Regression
Let's fit a linear regression model to predict the bitcoin price on the 10th day from the previous 9 days.

In [ ]:
import time

def loss(pred, y):
    """Mean Absolute Error loss"""
    return tf.reduce_mean(tf.abs(pred - y))

def train(model, loss, dataloader, optimizer):
    total_error = 0.
    for i, sequences in enumerate(dataloader):
        price_history = ??
        price_now = sequences[:, -1]
        target = ??

        with tf.GradientTape() as tape:
            pred_now = ??
            l = loss(pred_now, target)
        total_error += l.numpy()
        grads = tape.gradient(l, model.trainable_variables)

        optimizer.apply_gradients(zip(??, model.trainable_variables))
    return total_error/i

def fit(model, loss, dataloader, epochs=30):
    optimizer = tf.optimizers.Adam(learning_rate=lr)
    for ep in range(epochs):
        err = train(model, loss, dataloader, optimizer)
        print(f'[Ep{ep} Error {err:.3f}]')

In [ ]:
price_history_len = 9
lr = 0.0005
batch_size = 32
epochs = 300

model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(price_history_len,), use_bias=False)
])

dataset = TimeSeriesDataset(price.values, sequence_length=price_history_len + 1)
dataloader = tf.data.Dataset.from_generator(dataset, output_signature=(tf.TensorSpec(shape=(10,), dtype=tf.float32))).batch(batch_size)

In [ ]:
fit(model, loss, dataloader, epochs=epochs)

In [ ]:
predictions, errors = [], []
for i in range(len(dataset)):
    sequence = dataset[i]
    past, now = ??
    pred = ??
    err = pred - now
    errors.append(err)
    predictions.append(pred)

In [ ]:
plt.plot([None] * 9 + predictions, label='prediction')
plt.plot(price, label='ground truth')
plt.ylabel('Bitcoin Price [$]')
plt.xticks(range(0, len(date), 300), date.iloc[::300], rotation=30)
plt.legend()
plt.show()

In [ ]:
plt.hist(np.array(errors), bins=20, edgecolor='black')
plt.xlabel('Error [$]')
plt.ylabel('Frequency')
plt.title('Histogram of Errors')
plt.show()